# Exploring Hackers News Posts
In this project, we'll compare two different types of posts from Hacker News, a popular site where technology related stories (or 'posts') are voted and commented upon. The two types of posts we'll explore begin with either Ask HN or Show HN.

Users submit Ask HN posts to ask the Hacker News community a specific question, such as "What is the best online course you've ever taken?" Likewise, users submit Show HN posts to show the Hacker News community a project, product, or just generally something interesting.

We'll specifically compare these two types of posts to determine the following:

Do Ask HN or Show HN receive more comments on average?
Do posts created at a certain time receive more comments on average?
It should be noted that the data set we're working with was reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions.

# Introduction
First, we'll read in the data and remove the headers.

In [1]:
# Read in the data
from csv import reader
opened_file = open('hacker_news.csv')
read_file = reader(opened_file)
hn = list(read_file)
# Remove the headers
headers = hn[0]
hn = hn[1:]

In [2]:
headers

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']

In [3]:
hn[:4]

[['12224879',
  'Interactive Dynamic Video',
  'http://www.interactivedynamicvideo.com/',
  '386',
  '52',
  'ne0phyte',
  '8/4/2016 11:52'],
 ['10975351',
  'How to Use Open Source and Shut the Fuck Up at the Same Time',
  'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/',
  '39',
  '10',
  'josep2',
  '1/26/2016 19:30'],
 ['11964716',
  "Florida DJs May Face Felony for April Fools' Water Joke",
  'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/',
  '2',
  '1',
  'vezycash',
  '6/23/2016 22:20'],
 ['11919867',
  'Technology ventures: From Idea to Enterprise',
  'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
  '3',
  '1',
  'hswarna',
  '6/17/2016 0:01']]

We can see above that the data set contains the title of the posts, the number of comments for each post, and the date the post was created. Let's start by exploring the number of comments for each type of post.

# Extracting Ask HN and Show HN Posts
First, we'll identify posts that begin with either Ask HN or Show HN and separate the data for those two types of posts into different lists. Separating the data makes it easier to analyze in the following steps.

In [4]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    if title.lower().startswith('ask hn'):
        ask_posts.append(row)
    elif title.lower().startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)

print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))

1744
1162
17194


In [5]:
ask_posts[:3]

[['12296411',
  'Ask HN: How to improve my personal website?',
  '',
  '2',
  '6',
  'ahmedbaracat',
  '8/16/2016 9:55'],
 ['10610020',
  'Ask HN: Am I the only one outraged by Twitter shutting down share counts?',
  '',
  '28',
  '29',
  'tkfx',
  '11/22/2015 13:43'],
 ['11610310',
  'Ask HN: Aby recent changes to CSS that broke mobile?',
  '',
  '1',
  '1',
  'polskibus',
  '5/2/2016 10:14']]

In [6]:
show_posts[:4]

[['10627194',
  'Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform',
  'https://iot.seeed.cc',
  '26',
  '22',
  'kfihihc',
  '11/25/2015 14:03'],
 ['10646440',
  'Show HN: Something pointless I made',
  'http://dn.ht/picklecat/',
  '747',
  '102',
  'dhotson',
  '11/29/2015 22:46'],
 ['11590768',
  'Show HN: Shanhu.io, a programming playground powered by e8vm',
  'https://shanhu.io',
  '1',
  '1',
  'h8liu',
  '4/28/2016 18:05'],
 ['12178806',
  'Show HN: Webscope  Easy way for web developers to communicate with Clients',
  'http://webscopeapp.com',
  '3',
  '3',
  'fastbrick',
  '7/28/2016 7:11']]

In [7]:
other_posts[:3]

[['12224879',
  'Interactive Dynamic Video',
  'http://www.interactivedynamicvideo.com/',
  '386',
  '52',
  'ne0phyte',
  '8/4/2016 11:52'],
 ['10975351',
  'How to Use Open Source and Shut the Fuck Up at the Same Time',
  'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/',
  '39',
  '10',
  'josep2',
  '1/26/2016 19:30'],
 ['11964716',
  "Florida DJs May Face Felony for April Fools' Water Joke",
  'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/',
  '2',
  '1',
  'vezycash',
  '6/23/2016 22:20']]

# Calculating the Average Number of Comments for Ask HN and Show HN Posts
Now that we separated ask posts and show posts into different lists, we'll calculate the average number of comments each type of post receives.

In [8]:
total_ask_comments = 0
for row in ask_posts:
    total_ask_comments += float(row[4])
    
ave_ask_comments = total_ask_comments / len(ask_posts)
print(ave_ask_comments)

14.038417431192661


In [9]:
total_show_comments = 0
for row in show_posts:
    total_show_comments += float(row[4])
    
ave_show_comments = total_show_comments / len(show_posts)
print(ave_show_comments)

10.31669535283993


On average, ask posts in our sample receive approximately 14 comments, whereas show posts receive approximately 10. Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts.

# Finding the Amount of Ask Posts and Comments by Hour Created
Next, we'll determine if we can maximize the amount of comments an ask post receives by creating it at a certain time. First, we'll find the amount of ask posts created during each hour of day, along with the number of comments those posts received. Then, we'll calculate the average amount of comments ask posts created at each hour of the day receive.

In [10]:
import datetime as dt
result_list = []
for row in ask_posts:
    result_list.append([row[6], float(row[4])])

counts_by_hour = {}
comments_by_hour = {}
date_format = "%m/%d/%Y %H:%M"
#strptime 文字列の日付をフォーマットにしたがって日付に変換
#strftime 日付を文字列に変換
for row in result_list:
    date = row[0]
    n_comments = row[1]
    time = dt.datetime.strptime(date, date_format).strftime('%H')
    if time in counts_by_hour:
        counts_by_hour[time] += 1
        comments_by_hour[time] += n_comments
    else:
        counts_by_hour[time] = 1
        comments_by_hour[time] = n_comments
    
comments_by_hour

{'00': 447.0,
 '01': 683.0,
 '02': 1381.0,
 '03': 421.0,
 '04': 337.0,
 '05': 464.0,
 '06': 397.0,
 '07': 267.0,
 '08': 492.0,
 '09': 251.0,
 '10': 793.0,
 '11': 641.0,
 '12': 687.0,
 '13': 1253.0,
 '14': 1416.0,
 '15': 4477.0,
 '16': 1814.0,
 '17': 1146.0,
 '18': 1439.0,
 '19': 1188.0,
 '20': 1722.0,
 '21': 1745.0,
 '22': 479.0,
 '23': 543.0}

# Calculating the Average Number of Comments for Ask HN Posts by Hour

In [11]:
avg_by_hour = []
for time in comments_by_hour:
    avg_by_hour.append([time, comments_by_hour[time] / counts_by_hour[time]])
avg_by_hour

[['16', 16.796296296296298],
 ['06', 9.022727272727273],
 ['21', 16.009174311926607],
 ['05', 10.08695652173913],
 ['03', 7.796296296296297],
 ['04', 7.170212765957447],
 ['09', 5.5777777777777775],
 ['15', 38.5948275862069],
 ['12', 9.41095890410959],
 ['17', 11.46],
 ['07', 7.852941176470588],
 ['11', 11.051724137931034],
 ['23', 7.985294117647059],
 ['20', 21.525],
 ['13', 14.741176470588234],
 ['10', 13.440677966101696],
 ['22', 6.746478873239437],
 ['00', 8.127272727272727],
 ['14', 13.233644859813085],
 ['08', 10.25],
 ['19', 10.8],
 ['01', 11.383333333333333],
 ['02', 23.810344827586206],
 ['18', 13.20183486238532]]

# Sorting and Printing Values from a List of Lists

In [12]:
swap_avg_by_hour = []
for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])
    
sorted_swap = sorted(swap_avg_by_hour, reverse =True)

In [13]:
print('Top 5 Hours for Ask Posts Comments')

for avg, time in sorted_swap[:5]:
    print(
        '{}: {:.2f} average comment per post' .format(
            dt.datetime.strptime(time,'%H').strftime('%H:%M'), avg
        )
    ) 

Top 5 Hours for Ask Posts Comments
15:00: 38.59 average comment per post
02:00: 23.81 average comment per post
20:00: 21.52 average comment per post
16:00: 16.80 average comment per post
21:00: 16.01 average comment per post


The hour that receives the most comments per post on average is 15:00, with an average of 38.59 comments per post. There's about a 60% increase in the number of comments between the hours with the highest and second highest average number of comments.

According to the data set documentation, the timezone used is Eastern Time in the US. So, we could also write 15:00 as 3:00 pm est.

# Conclusion
In this project, we analyzed ask posts and show posts to determine which type of post and time receive the most comments on average. Based on our analysis, to maximize the amount of comments a post receives, we'd recommend the post be categorized as ask post and created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est).

However, it should be noted that the data set we analyzed excluded posts without any comments. Given that, it's more accurate to say that of the posts that received comments, ask posts received more comments on average and ask posts created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est) received the most comments on average.

# Next steps
* Determine if show or ask posts receive more points on average.
* Determine if posts created at a certain time are more likely to receive more points.
* Compare your results to the average number of comments and points other posts receive.
* Use Dataquest's data science project style guide to format your project.

In [14]:
#Determine if show or ask posts receive more points on average
total_ask_points = 0
for row in ask_posts:
    total_ask_points += int(row[3])

avg_ask_points = total_ask_points / len(ask_posts)
print(avg_ask_points)

15.061926605504587


In [15]:
total_show_points = 0
for row in show_posts:
    total_show_points += int(row[3])

avg_show_points = total_show_points / len(show_posts)
print(avg_show_points)

27.555077452667813


On average, show posts approximately 27 points , whereas as ask posts receive approximately 15 points. Since show posts are more likely to receive points, we'll focus our remaining analysis just on these posts.

In [16]:
#Determine if posts created at a certain time are more likely to receive more points.
result_points_list = []
for row in show_posts:
    result_points_list.append([row[-1],float(row[3])])
    
counts_by_hour = {}
points_by_hour = {}
for row in result_points_list:
    date = row[0]
    n_points = row[1]
    time = dt.datetime.strptime(date,date_format).strftime('%H')
    if time in counts_by_hour:
        counts_by_hour[time] += 1
        points_by_hour[time] += n_points
    else:
        counts_by_hour[time] = 1
        points_by_hour[time] = n_points
points_by_hour

{'00': 1173.0,
 '01': 700.0,
 '02': 340.0,
 '03': 679.0,
 '04': 386.0,
 '05': 104.0,
 '06': 375.0,
 '07': 494.0,
 '08': 519.0,
 '09': 553.0,
 '10': 681.0,
 '11': 1480.0,
 '12': 2543.0,
 '13': 2438.0,
 '14': 2187.0,
 '15': 2228.0,
 '16': 2634.0,
 '17': 2521.0,
 '18': 2215.0,
 '19': 1702.0,
 '20': 1819.0,
 '21': 866.0,
 '22': 1856.0,
 '23': 1526.0}

In [17]:
#Calculating the Average Number of Points for Show HN Posts by Hour
avg_points_by_hour = []
for time in points_by_hour:
    avg_points_by_hour.append([time, points_by_hour[time] / counts_by_hour[time]])
avg_points_by_hour

[['16', 28.322580645161292],
 ['06', 23.4375],
 ['05', 5.473684210526316],
 ['21', 18.425531914893618],
 ['03', 25.14814814814815],
 ['04', 14.846153846153847],
 ['09', 18.433333333333334],
 ['15', 28.564102564102566],
 ['12', 41.68852459016394],
 ['17', 27.107526881720432],
 ['07', 19.0],
 ['11', 33.63636363636363],
 ['23', 42.388888888888886],
 ['20', 30.316666666666666],
 ['13', 24.626262626262626],
 ['10', 18.916666666666668],
 ['22', 40.34782608695652],
 ['00', 37.83870967741935],
 ['14', 25.430232558139537],
 ['08', 15.264705882352942],
 ['19', 30.945454545454545],
 ['01', 25.0],
 ['18', 36.31147540983606],
 ['02', 11.333333333333334]]

In [18]:
#Sorting and Printing Values from a List of Lists
swap_avg_points_by_hour = []
for row in avg_points_by_hour:
    swap_avg_points_by_hour.append([row[1], row[0]])
    
sorted_swap = sorted(swap_avg_points_by_hour, reverse = True)

In [19]:
print('Top 5 Hours for Show Posts Points')

for avg, time in sorted_swap[:5]:
    print(
        '{}: {:.2f} average points per post' .format(
        dt.datetime.strptime(time,'%H').strftime('%H:%M'),avg
        )
    )

Top 5 Hours for Show Posts Points
23:00: 42.39 average points per post
12:00: 41.69 average points per post
22:00: 40.35 average points per post
00:00: 37.84 average points per post
18:00: 36.31 average points per post


The hour that receives the most points per post on average is 23:00, with an average of 42.39 points per post. 
According to the data set documentation, the timezone used is Eastern Time in the US. So, we could also write 23:00 as 11:00 pm est.

In [26]:
# Compare your results to the average number of comments and points other posts receive
total_other_comments = 0
total_other_points = 0
for row in other_posts:
    total_other_comments += float(row[4])
    total_other_points += float(row[3])

avg_other_comments = total_other_comments / len(other_posts)
avg_other_points = total_other_points / len(other_posts)

print('avg_other_comments: ',avg_other_comments)
print('avg_other_points: ',avg_other_points)

avg_other_comments:  26.8730371059672
avg_other_points:  55.4067698034198


In [56]:
from statistics import mean
other_comments = []
other_points = []
for row in other_posts:
    other_comments.append(float(row[4])) 
    other_points.append(float(row[3]))
print(max(other_comments), mean(other_comments))
print(max(other_points), mean(other_points))

1733.0 26.8730371059672
2553.0 55.4067698034198


What can I say?
